# Intro

Divvy is a bike share program based in Chicago (https://www.divvybikes.com/). With its app users can pay to use Divvy bikes from its 600+ stations across Chicago city. 

The dataset contains 9 million riding trips that are powered by Divvy from 2014 to 2017. For each trip we know the start timestamp, the geogprahic locations of the start and end stations, temperature and weather conditions.

The goal of this project is to detect patterns in the usage of bike sharing servcies. By exploring the dataset we've managed to find something interesting:

🗓 **Monthly trend:** the usage of bike sharing services has a peak in summer seasons, i.e. from June to August.  

💼 **DOW trend:** in general people are more likely to ride shared bikes on weekdays thab over weekend.

⏱ **Hourly trend:** we see different patterns on weekdays vs weekends. From Monday to Friday, the peak hours of Divvy services match with the commuting rush hours, i.e. 7am - 9am and 5pm - 6pm. On weekends the hourly patter is rather smooth without significant patterns, the usage is higher in the day than night.

🗺 **Geo pattern:** by plotting all stations on the map and coloring them with "heat" (i.e. the total number of bike sharing trips starting from the station), we find that the closer the station is to the city center, the more biking sharing trips satrt from the station.

⏱/🗺 **Hourly geo parttern:** the centralisation of heat areas in city center is most obvious during peak hours (i.e. 7am - 9am and 5pm - 7pm). During other slots, the geo difference in traffic is less significant.

♨️ **Temperatue:** there is a positive correlation beween temperature and the usage of biking riding service during daytime and eveninig (i.e. from 8am to approx. 10pm).

🌦 **Weather conditions:** as one may expect, people are less likely to use bike sharing service in bad weathers like rain/snow and thunderstorm.

In addition, we compare three models to predict the number of trips starting from a specific station during a specific hour. The XGBoost regress gives the best model (with the highest R2 score and the lowest MSE) and suggests the top3 most importan features are: months, whether the station is among the top10% in terms of heat (measured by the number of trips from the stations in 2017) and hours.

Suggestion to further improve the prediction model: one can import the event calendar of Chicago city with must-have information like time, location and audience size and include this in the prediction model.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Step 0: Load dataset
import pandas as pd
from time import time
t0 = time()
data = pd.read_csv("/kaggle/input/chicago-divvy-bicycle-sharing-data/data.csv")
print("Done in %0.3fs." % (time() - t0))

# Data Pre-processing


In [ ]:
# Step 1: Data pre-processing
# 1.1 First look into the dataset
data.head()

In [ ]:
print("The dataset contains {0[0]: ,.0f} rows and {0[1]: .0f} variables.".format(data.shape))

In [ ]:
data.info() # There is no missing value in the dataset

In [ ]:
# 1.2 Distribution across the years
import matplotlib.pyplot as plt
plt.style.use("seaborn")
fig = data["year"].value_counts().sort_index().plot(kind="bar")
plt.title("Number of Trips in Each Year")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
year_month_pivot = pd.pivot_table(data, index="year", columns="month", values="trip_id", aggfunc="count")
fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(year_month_pivot)
plt.title("Number of Trips per Year and per Month")

In [ ]:
# 1.2 Create weekend column
t0 = time()
data["day"].value_counts() # Confirm that this column indicates the day of week
data[["starttime", "day"]].head() # Confirm the coding of DOW: 0 = Monday
dowmapping = pd.DataFrame.from_dict({"0": "Monday", "1": "Tuesday", "2": "Wednesday", 
                                     "3": "Thursday", "4": "Friday", "5": "Saturday", 
                                     "6": "Sunday"}, 
                                    orient="index")
dowmapping.reset_index(inplace=True)
dowmapping.columns = ["num_coding", "dow"]
dowmapping["num_coding"] = dowmapping["num_coding"].astype("int64")
data = pd.merge(data, dowmapping, how="left", left_on="day", right_on="num_coding")
del(data["num_coding"])
data["weekend"] = data["day"].apply(lambda x: x in (5, 6))
print("Done in %0.3fs." % (time()-t0))

In [ ]:
# 1.3 Create date column
# This process can take 15 minutes.
# t0 = time()
# from datetime import datetime
# data["starttime"] = data["starttime"].astype("str")
# data["starttime"] = data["starttime"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
# data["startdate"] = data["starttime"].apply(lambda x: x.date())
# data["starthour"] = data["starttime"].apply(lambda x: x.replace(minute=0))
# data["startmonth"] = data["startdate"].apply(lambda x: x.replace(day=1))
# # Since this step takes 15 minutes to complete this step I will save the file to avoid long runtime.
# data.to_csv("data_preprocess.csv")
# print("Done in %0.3fs." % (time()-t0))

In [ ]:
# data = pd.read_csv("data_preprocess.csv")

In [ ]:
# 1.4 Creat starthour pivot table for prediction modelling
# starthour_pivot = pd.pivot_table(data=data,
#                                  index=["starthour", 
#                                    "year", "month", "day", "hour", "weekend", "startdate",
#                                    "temperature", "events", 
#                                    "from_station_id", "from_station_name"], 
#                                  values="trip_id", aggfunc="count")
# starthour_pivot = pd.DataFrame(starthour_pivot)
# starthour_pivot.to_csv("starthour_pivot.csv")

# Explanatory Data Analysis

In [ ]:
data.describe(include="all")

## Temporal Trends
### Monthly Trend

Overall the biking sharing service provided by Divvy has been more and more popular over the years. Meanwhile, as one can expect, summmer time (June, July and August) is the peak season for bike riding trips.

In [ ]:
# Step 2: Explanatory analysis
# 2.1 Temporal trends
# Monthly trend
year_month = pd.DataFrame(data.groupby(["year", "month"]).count()["trip_id"])
year_month.reset_index(inplace=True)
year_month.columns = ["year", "month", "count_trips"]
g = sns.catplot(x="month", y="count_trips", col="year", data=year_month, kind="bar")
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of Trips per Month')
plt.show()

### DOW Trend

We also observe that people are less likely to user Divvy biking serivce on weekends.

In [ ]:
# DOW trend
year_day = pd.DataFrame(data.groupby(["year", "day"]).count()["trip_id"])
year_day.reset_index(inplace=True)
year_day.columns = ["year", "day", "count_trips"]
g = sns.catplot(x="day", y="count_trips", col="year", data=year_day, kind="bar")
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of Trips per DOW (0=Monday)')
plt.show()

### Hourly Trend

On weekdays the peak matches the rush hours: 8am and 5pm. On weekends the hourly trends are much smoother: the usage increased steadily since 9am and peaked at 12pm, then descreased gradually.

Therefore, in later steps we would split weekends from weekdays.

In [ ]:
# Hourly trend
day_hour = pd.DataFrame(data[data["year"]==2017].groupby(["day", "hour"]).count()["trip_id"])
day_hour.reset_index(inplace=True)
day_hour.columns = ["day", "hour", "count_trips"]
g = sns.catplot(x="hour", y="count_trips", col="day", data=day_hour, kind="bar")
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of Trips per Hour (day=0 means Monday)')
plt.show()

In [ ]:
# 2.2 Geo patterns
# Import packages
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Generate the list of stations
start = data[["latitude_start", "longitude_start", "from_station_name"]]
start.columns = ["lat", "lon", "station_name"]
end = data[["latitude_end", "longitude_end", "to_station_name"]]
end.columns = ["lat", "lon", "station_name"]
station_info = (pd.concat([start, end])).groupby("station_name").mean()[["lat", "lon"]] # One station name can be mapped with multiple pairs of latitude and longitude
station_info.reset_index(inplace=True)
station_info.columns = ["station_name", "lat", "lon"]

station_start_trips = pd.DataFrame(data[data["year"]==2017]["from_station_name"].value_counts())
station_start_trips.reset_index(inplace=True)
station_start_trips.columns = ["station_name", "count_start_trips_2017"]
station_info = station_info.merge(station_start_trips, how="left")

station_end_trips = pd.DataFrame(data[data["year"]==2017]["to_station_name"].value_counts())
station_end_trips.reset_index(inplace=True)
station_end_trips.columns = ["station_name", "count_end_trips_2017"]
station_info = station_info.merge(station_end_trips, how="left")

station_info.info()

In [ ]:
station_info = station_info.dropna()

In [ ]:
station_info["count_start_trips_2017"].describe()

In [ ]:
quantiles = []
levels = []
for i in range(11):
    quantiles.append(np.percentile(station_info["count_start_trips_2017"], i*10))
    levels.append("level_"+str(i+1))
station_info["count_start_trips_intervals"] = pd.cut(station_info["count_start_trips_2017"], quantiles, labels=levels[:-1])
station_info[station_info["count_start_trips_2017"]==3]["count_start_trips_intervals"] = "level_1"
station_info = station_info.sort_values(by="count_start_trips_2017", ascending=True)

### Geographic Pattern

As one may expect, the closer the station is to the city center, the more riding trips come from the station.

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x="lon", y="lat", hue="count_start_trips_intervals", palette="RdBu",
                data=station_info, alpha=0.5)
plt.title("Scatter Plot of Stations that were active in 2017 (colored by heat)")

In [ ]:
# Plot all stations on Chicago map
# px.set_mapbox_access_token("XXX") # Replace XXX with your Mapbox Token
# fig = px.scatter_mapbox(station_info.sort_values(by="count_start_trips_2017", ascending=True),
#                         lat="lat", 
#                         lon="lon",
#                         color="count_start_trips_2017",
#                         color_continuous_scale=px.colors.sequential.Plasma,
#                         text="station_name",
#                         opacity=0.7, 
#                         zoom=10)
# fig.show()

From the cum. density we can see that 200 out of 592 stations account for 80% of all bike riding trips. 

In [ ]:
station_info = station_info.sort_values(by="count_start_trips_2017", ascending=False)
station_info["count_start_trips_cumsum"] = station_info["count_start_trips_2017"].cumsum()
(station_info["count_start_trips_cumsum"].reset_index(drop=True) / sum(station_info["count_start_trips_2017"])).plot()
plt.title("Cum. Density of Number of Trips Starting from Each Station")
plt.show()

### Hourly Geographic Pattern

The animation above shows that the overall coverage across the city may vary from hour to hour. During peake hours the city center always has the highest usage.

In [ ]:
# Plot an animation of scatter plot to see which are the most popular start stations in each hour
start_station_hour = data[data["year"]==2017].groupby(["from_station_name", "hour"]).count()["trip_id"].reset_index()
start_station_hour = start_station_hour.merge(station_info[["station_name", "lat", "lon"]], how="left", left_on="from_station_name", right_on="station_name")
del(start_station_hour["from_station_name"])
start_station_hour = start_station_hour.rename({"trip_id": "count_start_trips"}, axis="columns")

In [ ]:
for hour in range(24):
    # Slice the whole dataframe
    station_start_trips_per_hour = start_station_hour[start_station_hour["hour"]==hour]
    # Take the hourly max
    hourly_max = station_start_trips_per_hour["count_start_trips"].max()
    # Generate labels
    start_station_hour.loc[start_station_hour["hour"]==hour, "start_trips_heat_per_hour"] = start_station_hour.loc[start_station_hour["hour"]==hour, "count_start_trips"].apply(lambda x: x/hourly_max)

In [ ]:
fig = px.scatter(start_station_hour.sort_values(by=["hour", "count_start_trips"], ascending=[True, True]),
                 x="lon", y="lat", 
                 animation_frame="hour",
                 color="start_trips_heat_per_hour", 
                 opacity=0.5,
                 size="count_start_trips", size_max=60,
                 range_x=[-87.9,-87.5], range_y=[41.7, 42.1])
fig.update_layout(title="Hourly Start Trips from Each Station (Year 2017)")
fig.show()

In [ ]:
station_info.info()

In [ ]:
starthour = pd.read_csv("/kaggle/input/starthour-pivot/starthour_pivot.csv")
starthour = starthour.rename({"trip_id": "count_start_trips"}, axis="columns")
starthour.head()

In [ ]:
print("We have {0[0]: ,.0f} records of hourly number of trips starting from each station.".format(starthour.shape))

In [ ]:
# 2.3 Temparature
starthour["temperature"].hist()
plt.title("Histogram of Temperature")
plt.show()

In [ ]:
starthour.head(10)

From the first 20 records we can see that temperature and events (i.e. weather conditions) are aggregated on hourly level across all stations. Therefore, to reduce run time we will generate a new pivot table only at hourly level and sum up trips from all stations during that hour.

In [ ]:
starthour["events"].value_counts()

We find some events have value "unknown" and try to fill the missing value with approximate values.

### Temperature

In popular hours we can observe that a positive correation between temparature and the usage of bike sharing service.

In [ ]:
# from datetime import datetime
unknown_events = starthour[starthour["events"]=="unknown"]
unknown_events.groupby(["starthour"]).count()

In [ ]:
# Check the weather condition on "201x-04-18 11:00:00"
starthour[starthour["starthour"].apply(lambda x: x in ["2015-04-18 11:00:00", "2016-04-18 11:00:00", "2017-04-18 11:00:00"])]                                               

In [ ]:
# Fill the weather condition for "2014-04-18 11:00:00" as "cloudy"
starthour.loc[starthour["starthour"]=="2014-04-18 11:00:00", "events"] = "cloudy"

In [ ]:
# Check the weather condition on "201x-06-30 10:00:00"
starthour[starthour["starthour"].apply(lambda x: x in ["2014-06-30 10:00:00", "2015-06-30 10:00:00", "2017-06-130 10:00:00"])]                                                                 

In [ ]:
# Fill the weather condition for "2016-06-30 10:00:00" as "cloudy"
starthour.loc[starthour["starthour"]=="2016-06-30 10:00:00", "events"] = "cloudy"

In [ ]:
starthour_agg = pd.pivot_table(starthour,
                               index=['starthour', 'year', 'hour', 'temperature', 'events'],
                               values=['count_start_trips'], 
                               aggfunc=np.sum)
starthour_agg.reset_index(inplace=True)

In [ ]:
hour_temp_sum_trips_2017 = pd.pivot_table(starthour_agg[starthour_agg["year"]==2017],
                                          index=['hour', 'temperature'],
                                          values=['starthour', 'count_start_trips'], 
                                          aggfunc={"starthour": np.count_nonzero, "count_start_trips": np.sum})
hour_temp_sum_trips_2017.reset_index(inplace=True)
hour_temp_sum_trips_2017["avg_trips"] = hour_temp_sum_trips_2017["count_start_trips"] / hour_temp_sum_trips_2017["starthour"]

In [ ]:
g = sns.FacetGrid(hour_temp_sum_trips_2017,
                  col="hour", col_wrap=4,
                  margin_titles=True)
g.set(ylim=(0, 60))
g = g.map(plt.scatter, "temperature", "avg_trips", s=25, color="c", alpha=0.3)

In [ ]:
# 2.4 Weather conditions
hour_event_sum_trips_2017 = pd.pivot_table(starthour_agg[starthour_agg["year"]==2017],
                                           index=['hour', 'events'],
                                           values=['starthour', 'count_start_trips'], 
                                           aggfunc={"starthour": np.count_nonzero, 
                                                    "count_start_trips": np.sum})
hour_event_sum_trips_2017.reset_index(inplace=True)
hour_event_sum_trips_2017["avg_trips"] = hour_event_sum_trips_2017["count_start_trips"] / hour_event_sum_trips_2017["starthour"]

### Weather Conditions

As expected, people are less likely to ride by bike in case of bad weather conditions like rain, snow or thunderstorms.

In [ ]:
g = sns.FacetGrid(hour_event_sum_trips_2017,
                  col="hour", col_wrap=4, hue="events",
                  height=4, aspect=1.5,
                  margin_titles=True)
g.set(ylim=(0, 60))
g = g.map(plt.bar, "events", "avg_trips")

# Predict number of trips in a specific hour

In this section we will build a model to predict the number of trips in a specifc hour.

In [ ]:
starthour.info()

In [ ]:
starthour.head()

In [ ]:
starthour2017 = starthour[starthour["year"]==2017]
starthour2017 = starthour2017.merge(station_info[["station_name", "lat", "lon", "count_start_trips_intervals"]], 
                                    how = "left",
                                    left_on ="from_station_name", right_on ="station_name")
del(starthour)

In [ ]:
starthour2017.head()

In [ ]:
# Step 3: Run prediction model
# 3.1 Create dummy variables
weekend = pd.get_dummies(starthour2017["weekend"], prefix="weekend", drop_first=True)
events = pd.get_dummies(starthour2017["events"], prefix="event", drop_first=True)
station_heat = pd.get_dummies(starthour2017["count_start_trips_intervals"], prefix="station_heat", drop_first=True)
X = starthour2017[["month", "day", "hour", "temperature", "lat", "lon"]]
for cols in [weekend, events, station_heat]:
    X = pd.concat([X, cols], axis=1)
    del(cols)

In [ ]:
# 3.2 Split train and test set
from sklearn.model_selection import train_test_split
y = starthour2017[['count_start_trips']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
del(X)
del(y)

### Feature Importance

We test three models on our test set: linear regression, Gradient Boosting Regressor and XGBoost Regressor (with highest R2 score and lowest MSE).

The XGBoost mdoel suggests that the top3 features with highest explaining power are: months, whether the station is among the top10% stations in terms of "heat" and hours.

In [ ]:
# 3.4 Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
lr = LinearRegression()
lr.fit(X_train, y_train)
y_test_pred = lr.predict(X_test)
print("The R-square score of linear model on test set is {0: .4f}.".format(r2_score(y_test, y_test_pred)))
print("The MSE of linear model on test set is {0: .4f}.".format(mean_squared_error(y_test, y_test_pred)))

In [ ]:
# 3.5 GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
# gbr = GradientBoostingRegressor()
# parameters = {
#     'n_estimators': [100, 120, 140],
#     'max_depth': [3, 5, 7]
# }
# grid_search = GridSearchCV(estimator=gbr, param_grid=parameters, cv=3, n_jobs=1)
# grid_search.fit(X_train, y_train)
# print("The best parameters are: \n")
# print(grid_search.best_params_)
t0 = time()
gbr = GradientBoostingRegressor(n_estimators=100, max_depth=3)
gbr.fit(X_train, y_train)
print("Done in %0.3fs." % (time()-t0))
y_test_pred = gbr.predict(X_test)
print("The R-square score of GBR model on test set is {0: .4f}.".format(r2_score(y_test, y_test_pred)))
print("The MSE of GBR model on test set is {0: .4f}.".format(mean_squared_error(y_test, y_test_pred)))
gbr_feature_inportance = pd.DataFrame(gbr.feature_importances_, index=X_train.columns)
gbr_feature_inportance.sort_values(by=0, ascending=True).tail(10).plot(kind="barh")
plt.title("Feature Importance by GBR Model")
plt.show()

In [ ]:
# 3.3 XGBoost
import xgboost as xgb
t0 = time()
xgb_reg = xgb.XGBRegressor(n_estimators=100, max_depth=3)
xgb_reg.fit(X_train, y_train)
print("Done in %0.3fs." % (time()-t0))
y_test_pred = xgb_reg.predict(X_test)
print("The R-square score of XGBoost model on test set is {0: .4f}.".format(r2_score(y_test, y_test_pred)))
print("The MSE of XGBoost model on test set is {0: .4f}.".format(mean_squared_error(y_test, y_test_pred)))
xgb_feature_inportance = pd.DataFrame(xgb_reg.feature_importances_, index=X_train.columns)
xgb_feature_inportance.sort_values(by=0, ascending=True).tail(10).plot(kind="barh")
plt.title("Feature Importance by XGBoost Model")
plt.show()

# Appendix

Intro to Animation in Plotly: https://plot.ly/python/animations/

Clustering to Reduce Spatial Data Set Size: https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

Seaborn 
* Plot color code: https://seaborn.pydata.org/generated/seaborn.color_palette.html#seaborn.color_palette
* Facet grid plot: https://seaborn.pydata.org/generated/seaborn.FacetGrid.html

XGBoost regressor: https://dask-ml.readthedocs.io/en/stable/modules/generated/dask_ml.xgboost.XGBRegressor.html